<!--img src="images\cloudflow.png" width=1400 /-->

# Introduction


## How to use this notebook
This example runs Sima in the cloud or locally based on input created in the [Create load case folders with unique Sima input](#loadcase_id) section. The code is only a pilot and only intended for testing. 

This note book should be run in the following way:
1. [Initialize workflow builder](#initialize), run this Python code once at every notebook start to set up basic settings. [Set up run specific parameters](#runspecific) section should be changed if you want to change workspace or switch between cloud and local run. 2. If you changed something in this section, remember to rerun  [Builder class](#builder).
2. [Prepare Sima Input](#prepare) sets up reading of the environment input file and build Sima input accordingly. If the input file changes OO needs to change this section.
3. [Run analysis in cloud](#run) shall be run each time a new analysis needs to be run.
4. [Run Wasim and Sestran](#runwasim) modify to your own postprocessing needs.


# Initialize workflow <a id='initialize'></a>
Run only once when notebook is opened.

## Set up fixed user parameters <a id='fixed'></a>
The below parameters should only be needed to modify once.

In [3]:
from dnv.oneworkflow.config import WorkflowRunnerConfiguration
toolsPath = r'C:\Users\kblu\AppData\Local\OneCompute\\'
serviceExecutablePath = f"{toolsPath}DNV.One.Workflow.Hosts.LocalWorkflowHost\\wc.exe"
worker_host_apps_path = f"{toolsPath}DNV.One.Workflow.WorkerHosts.OneWorkflowWorkerHost.Local"

#Sima path must be specified
sima_exe_path = r'C:\Program Files\DNV\Sima V4.4-00'         


tempFolderPath = r"c:\OneWorkflowTemp" #local directory for worker host, should be at root due to issues with long file names on Windows
runnerConfig = WorkflowRunnerConfiguration(service_executable_path=serviceExecutablePath, temp_folder_path=tempFolderPath, startup_wait_time=10, worker_host_apps_path=worker_host_apps_path)

## Set up custom user parameters <a id='custom'></a>

In [13]:
from dnv.oneworkflow.config import WorkspaceConfiguration
# local workspace, all results will be put here after local or cloud runs
workspacePath = r'C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\workspace'
# location of common files for all analysis, has to be below workspacePath
commonFilesDirectory = "Input"
loadcasesParentDirectory = "LoadCases" #location of individual runs below workspace
resultsDirectory = "Results"                                                                          #location of results, below workspace
workspaceId = "SE28"
workspaceConfig = WorkspaceConfiguration(workspaceId, workspacePath, commonFilesDirectory, loadcasesParentDirectory, resultsDirectory)

loadcase_file = "test_cases.xlsx"
wasim_input_file = "test_cases_wasim_input.xlsx"
stask_file = "SimaTemplate.stask"

# Set up run specific parameters <a id='runspecific'></a>

In [5]:
cloud_run = False

## Builder class <a id='builder'></a>
Run only once workbook is started or if some parameters above are changed.

In [6]:
from oneWorkflowToolBox import WorkflowBuilder

builder = WorkflowBuilder(workspaceConfig, runnerConfig, cloud_run)
#If running locally the code below will also start the local workflow host.
workflow_client = builder.oneworkflow_client
workspace = builder.oneworkflow_conf.workspace_config
commonfiles_folder = workspace.common_files_directory
workflow_client = builder.oneworkflow_client
if (cloud_run):
    workflow_client.login()


The temporary blob storage directory is: c:\OneWorkflowTemp\oc_9kb425s6_blob
The temporary jobs root directory is: c:\OneWorkflowTemp\oc__cmh1hca_jobs


## Upload common files for the job <a id='upload'></a>
This step uploads all common files in folder *commonFilesDirectory*  to the job. Only needed to run if new common files are to be uploaded or workspace changed.

In [5]:
from dnv.onecompute.directory_client import FileOptions
try:
    workflow_client.upload_common_files(FileOptions(
        # max_size_bytes=124_000,
        #patterns=["**/*.py","**/*.inp"],
        overwrite=True))
except Exception as e:
    print(e)
    print("Ignore this error message if the files are already present.")


Error: [WinError 10061] No connection could be made because the target machine actively refused it. Failed to upload files from C:\Users\kblu\source\repos\ImproveFlowSimaDemonstrator\TestExampleImproveFlow\SimpleDemo\Input.


## Prepare Sima input  <a id='prepare'></a>


### Create command line input for Sima
The function below will create the command line input that will be used to run Sima. 

In [7]:
import os
from typing import Any
from dnv.onecompute import FileSpecification

workspace = builder.oneworkflow_conf.workspace_config
commonfiles_folder = workspace.common_files_directory
def get_commands_inputs(
        stask_file: str,
        case: dict[str, Any])->dict[str, dict[str, Any]]:
    commands = dict[str, Any]()
    commands["--consoleLog"] = ""
    commands["--log-level"] = "ALL"
    commands["--data"] = "."
    commands["--import"] = dict(file=FileSpecification(sharedfolder=True,
                                directory=commonfiles_folder, filename=stask_file))
    commands["--run"] = dict(task="WorkflowTask",
                             workflow="ExampleWorkflow")

    return {"commands": commands, "inputs": case}

### Create load cases with unique Sima input <a id='loadcase_id'></a>
This code generates the input needed for Sima for the individual load cases. The input parameters to vary in the Sima run are give in the loadcasefile.

In [8]:
import pandas as pd
from dnv.sesam.sima_command import SimaCommand
from dnv.onecompute.flowmodel import ParallelWork
from dnv.oneworkflow import PythonCommand
def get_parallel_work(single_task: bool = False):
    """Returns a parallel processing unit. If the single_task parameter is set to True, the unit will only contain the first task. 
    If set to False, all tasks will be included. The single task option is primarily used for testing purposes."""
    os.chdir(workspace.workspace_path)
    load_cases_parent_folder_name = workspace.load_cases_parent_directory

    parallel_work = ParallelWork()
    parallel_work.work_items.clear()

    # Open environmental input file
    df_cases = pd.read_excel(os.path.join(workspacePath, loadcase_file), index_col=0)
    for loadcase_folder_name, case in df_cases.iterrows():
        load_case_folder = os.path.join(
            load_cases_parent_folder_name, loadcase_folder_name)
        result_folder_lc = os.path.join(workspace.results_directory, loadcase_folder_name)
        # Get SIMA commands and inputs 
        commands_inputs = get_commands_inputs(
             stask_file,  case.to_dict()
        )
        test_command = PythonCommand(
            directory=commonFilesDirectory,
            filename="test.py",
            working_dir=loadcase_folder_name,
        )
        # Create SimaCommand instance
        sima_cmd = SimaCommand(sima_exe_path)
        sima_cmd.commands = commands_inputs["commands"]
        sima_cmd.input = commands_inputs["inputs"]
        sima_cmd.sima_result_files =[
                "*-sima.lis",
                "variable*.inp",
                "*.log",
                "results.tda",
                "results.txt",
                "sima_*.res",
                "sys-sima.dat",
                "sima_*.bin",
                "key_sima_*.txt",
                "sima.*"
            ]
        sima_cmd.working_directory = load_case_folder

        # Add work item to ParallelWork instance
        parallel_work.add(sima_cmd, work_unit_id=loadcase_folder_name+"_ID").output_directory(result_folder_lc,
                                                                                         include_files=["**/*.txt", "**/*.tda", "**/*.bin", "**/*.log","**/sima.*"])
        if single_task == True:
            break
    return parallel_work

# Run analysis <a id='run'></a>
This code will fetch data from the blob storage created in the step above, and run all the job tasks. The code will wait for all tasks to complete before downloading the results.

In [10]:
from oneWorkflowToolBox import run_workflow_async
import json

"""Tests SIMA and Python commands"""
# Upload Input Files
workflow_client.upload_input_files()

# Create Parallel Work Unit and Job
work_unit = get_parallel_work(builder)
job = workflow_client.create_job(work_unit)

job_json = json.dumps(job, default=lambda o: o.encode(), indent=4)
#print(job_json)
# Run workflow
await run_workflow_async(job, workflow_client)


Error: load cases directory C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\LoadCases does not exist.


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\kblu\\source\\repos\\improveflowH4-workflow\\SE28ExampleSimaWasimSestra\\test_cases.xlsx'

# Run Wasim and Sestra <a id='runwasim'></a>

In [14]:
from dnv.onecompute.flowmodel import WorkUnit
from SesamUtilities import WasimAndSestraTaskCreator
import pandas as pd
from dnv.oneworkflow import  ParallelWork
from oneWorkflowToolBox import run_workflow_async
import json
import os
builder.workspace.results_directory = "Results"
load_cases = ["test001", "test002", "test003"]


os.chdir(workspacePath)
topSuperElement = 3
# due to 5 field width on Sestra cards we need to use as short name here
additionalTemplateParameters = {'FMT': topSuperElement}

def run_wasim_and_sestra_using_results_from_sima(
        workspace_builder: WorkflowBuilder,
        load_cases) -> ParallelWork:
    """Creates a parallel work unit"""
    df_cases = pd.read_excel(os.path.join(
        workspacePath, wasim_input_file), index_col=0)
    workspace_config = workspace_builder.workspace
    parallel_work_units = list[WorkUnit]()

    for casename, case in df_cases.iterrows():
        if not casename in load_cases:
            print("skipping " + casename)
            continue

        casedict = case.to_dict()
        
        load_case_result_files_dir = os.path.join(workspace_config.results_directory, casename)
      
        cmd = WasimAndSestraTaskCreator(
            load_case_result_files_dir, commonfiles_folder, casedict, additionalTemplateParameters).CreateTasks()
        work_unit = (
            WorkUnit(cmd, f"post_rerun_{casename}")
            .input_directory(load_case_result_files_dir)
            .output_directory(load_case_result_files_dir, include_files=["**/sima.*", "**/*.txt", "**/*.tda", "**/*.bin", "**/*.log", "**/*.inp", "**/*.lis", "**/*.mlg", "**/*.sin"])
        )
        parallel_work_units.append(work_unit)

    return ParallelWork(parallel_work_units)


work_unit = run_wasim_and_sestra_using_results_from_sima(
    builder, load_cases)
if not cloud_run:
    workflow_client.upload_common_files()
    workflow_client.upload_result_files()
job = workflow_client.create_job(work_unit)
job_json = json.dumps(job, default=lambda o: o.encode(), indent=4)
print(job_json)
await run_workflow_async(job, workflow_client)


Error: common files directory C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\Input does not exist.
Error: results directory C:\Users\kblu\source\repos\improveflowH4-workflow\SE28ExampleSimaWasimSestra\Results does not exist.
{
    "$type": "DNVGL.One.Compute.Core.FlowModel.Job, DNVGL.One.Compute.Core",
    "properties": {},
    "JobId": "fb3a6804-46ed-41b7-bacf-5f024b916d54",
    "ServiceName": "OneWorkflowWorkerHost",
    "PoolId": "",
    "Work": {
        "$type": "DNVGL.One.Compute.Core.FlowModel.ParallelWork, DNVGL.One.Compute.Core",
        "properties": {},
        "id": "886a51c0-2483-42f5-8171-acf508e6444c",
        "Parallel": true,
        "WorkItems": [
            {
                "$type": "DNVGL.One.Compute.Core.FlowModel.WorkUnit, DNVGL.One.Compute.Core",
                "properties": {},
                "id": "post_rerun_test001",
                "TaskRole": 0,
                "Data": {
                    "Version": 1,
                   